In [57]:
# tekst do weryfikacji działania NEL w spaCy
test_text = """Gdy car Iwan Groźny ustanowił kaprów z wyraźnym upoważnieniem do walki z polskimi, król, 
zaniepokojony wielce tą „rzeczą niezwykłą i niebywałą  za przodków jego”, jak określił to wystąpienie rywala 
na Bałtyku, przeciwstawił nowemu niebezpieczeństwu swą strażniczą armadę, ale jednocześnie  zdecydował się 
na dyplomatyczne kroki w Moskwie przez posła Michała Haraburdę, którego uzbroił w odpis carskiego listu 
przypowiedniego.
Zygmunt August stawiał także przed swą flotą zadanie obrony wybrzeża i portów w razie napadu nieprzyjacielskiego.
Chciał jej przeto użyć, o ileby  Gdańsk dał pomoc, do uderzenia na Duńczyków i opanowania ich okrętów  w czasie 
postoju floty pod admirałem Frankiem na redzie gdańskiej we wrześniu 1571 roku. 
"""

In [1]:
import os
import json
import sys
import random
from collections import Counter
from pathlib import Path
import spacy
from spacy.training import Example
from spacy.ml.models import load_kb
from spacy.util import minibatch, compounding

In [2]:
nlp = spacy.load("pl_core_news_lg")

Testowane będą dwie postacie występujące w wikidata.org: Zygmunt II August i Iwan Groźny

In [34]:
names_dict = desc_dict = {}

Identyfiktory Q każdej postaci zostały odnalezione w wikidata, do dalszego przetwarzania przygotowane zostały słowniki nazw i opisów postaci, w których kluczem są identyfikatory Q. 

In [35]:
names_dict['Q7996'] = 'Iwan Groźny'
names_dict['Q54058'] = 'Zygmunt August'
desc_dict['Q7996'] = 'car Rosji'
desc_dict['Q54058'] = 'król Polski i wielki książę litewski'

Tworzenie obietu bazy wiedzy 

In [36]:
from spacy.kb import KnowledgeBase
kb = KnowledgeBase(vocab=nlp.vocab, entity_vector_length=300)

In [37]:
for qid, desc in desc_dict.items():
    desc_doc = nlp(desc)
    desc_enc = desc_doc.vector
    kb.add_entity(entity=qid, entity_vector=desc_enc, freq=321)

Do bazy wiedzy dodawane są nazwiska postaci, na początek w takiej formie w jakiej występują w wikidata, 
takie wystąpienia mają pewną rozpoznawalność stąd wartość probablities = 1. 

In [38]:
for qid, name in names_dict.items():
    kb.add_alias(alias=name, entities=[qid], probabilities=[1])

W przypadku artykułu S.Bodniaka nie ma właściwie jakiejś wieloznaczności w encjach osób, Zygmunt August będzie zawsze tym Zygmuntem, królem Polski, Iwan będzie carem Rosji. Większym wyzwaniem byłoby gdyby w tekście występowałby np. Mateusz Scharping jako kaper i inna postać np. Ernest Scharping będącą np. niemieckim dyplomatą, wówczas dla encji bedących samym nawiskiem łączenie z identyfikatorami z wikidata.org musiałoby następować na podstawie kontekstu. Tu jednak nie ma takiej sytuacji, do bazy można natomiast dodać dodatkowe aliasy naszych postaci, również pewnie identyfikowane z elementami wikidata. 

In [39]:
kb.add_alias(alias="Iwan IV", entities=['Q7996'], probabilities=[1])
kb.add_alias(alias="Iwan Groźny", entities=['Q7996'], probabilities=[1])
kb.add_alias(alias="Zygmunt August", entities=['Q54058'], probabilities=[1])

17497991486119362077

In [40]:
output_dir = Path.cwd().parent / "output" / "nel"
kb.to_disk(output_dir / "kb_bodniak")
nlp.to_disk(output_dir / "nlp_bodniak")

Przygotowanie danych treningowych na podstawie próbki zdań z artykułu S. Bodniaka zawierającego wzmianki na temat analizowanych 2 postaci. Dane zawierają ok 30 zdań. Tak jak w przypadku anotacji dla trenowania modelu NER tu również anotacja została przeprowadzona w programie doccano, a dane wyjściowe z doccano wymagały przetworzenia na format oczekiwany przez spaCy (w tym przypadku z pliku .jsonl dane importowane są bezpośrednio do skryptu w pythonie). Ponieważ jednak doccano nie wspiera bezpośrednio anotacji pod kątem NEL, a przynajmniej nie widze takiej opcji, zastosowane zostało małe obejście problemu - ze względu na małą liczbę anotowanych postaci w roli etykiet użyto identyfiktorów QID postaci z wikidata. Interpretacją tak uzyskanych danych zajął się poniższy skrypt.  

In [41]:
dataset_file = Path.cwd().parent / "data" / "bodniak_nel.jsonl"
with open(dataset_file, "r", encoding="utf-8") as f:
    data = f.readlines()

dataset = []

for line in data:
    line = json.loads(line)

    text = labels = None
    if "text" in line:
        text = line["text"]
    if "label" in line:
        labels = line["label"]

    if text and labels:
        ents = []
        for start, end, label in labels:
            offset = (start, end)
            QID = label
            entity_label = 'persName'
            entities = [(offset[0], offset[1], entity_label)]
            links_dict = {QID: 1.0}
            dataset.append((text, {"links": {offset: links_dict}, "entities": entities}))

In [42]:
print(dataset[6])

('Otóż słaby stan zatrudnienia gdańskiej marynarki handlowej ułatwił niewątpliwie Zygmuntowi Augustowi tworzenie straży morskiej.', {'links': {(80, 100): {'Q54058': 1.0}}, 'entities': [(80, 100, 'persName')]})


Mając przygotowany do trenowania dataset należy go podzielić na cześć treingową i część walidującą, zwykle 20 % zbioru wystarcza do walidacji.

In [43]:
qids = names_dict.keys()

In [44]:
gold_ids = []
for text, annot in dataset:
    for span, links_dict in annot["links"].items():
        for link, value in links_dict.items():
            if value:
                gold_ids.append(link)

print(Counter(gold_ids))

Counter({'Q54058': 15, 'Q7996': 15})


In [45]:
train_dataset = []
test_dataset = []
for QID in qids:
    indices = [i for i, j in enumerate(gold_ids) if j == QID]
    max_ind = len(indices) - 3
    train_dataset.extend(dataset[index] for index in indices[0:max_ind])  # wszystkie poza ostatnimi dwoma do zbioru treningowego
    test_dataset.extend(dataset[index] for index in indices[max_ind:len(indices)])  # ostanie dwa do zbioru testowego
    
random.shuffle(train_dataset)
random.shuffle(test_dataset)

Przygotowanie do trenowania modelu

In [46]:
TRAIN_EXAMPLES = []
if "sentencizer" not in nlp.pipe_names:
    nlp.add_pipe("sentencizer")
sentencizer = nlp.get_pipe("sentencizer")
for text, annotation in train_dataset:
    example = Example.from_dict(nlp.make_doc(text), annotation)
    example.reference = sentencizer(example.reference)
    TRAIN_EXAMPLES.append(example)

In [47]:
entity_linker = nlp.add_pipe("entity_linker", config={"incl_prior": False}, last=True)
entity_linker.initialize(get_examples=lambda: TRAIN_EXAMPLES, kb_loader=load_kb(output_dir / "kb_bodniak"))

In [48]:
with nlp.select_pipes(enable=["entity_linker"]):   # tylko komponent entity linker
    optimizer = nlp.resume_training()
    for itn in range(500): 
        random.shuffle(TRAIN_EXAMPLES)
        batches = minibatch(TRAIN_EXAMPLES, size=compounding(4.0, 32.0, 1.001))
        losses = {}
        for batch in batches:
            nlp.update(
                batch,   
                drop=0.2,
                losses=losses,
                sgd=optimizer,
            )
        if itn % 50 == 0:
            print(itn, "Losses", losses)
print(itn, "Losses", losses)

0 Losses {'entity_linker': 4.341289937496185}
50 Losses {'entity_linker': 0.17442050576210022}
100 Losses {'entity_linker': 0.21613184611002603}
150 Losses {'entity_linker': 0.5162582596143086}
200 Losses {'entity_linker': 0.744226594765981}
250 Losses {'entity_linker': 0.3348371287186941}
300 Losses {'entity_linker': 0.15559369325637817}
350 Losses {'entity_linker': 0.49064404765764874}
400 Losses {'entity_linker': 0.3006354868412018}
450 Losses {'entity_linker': 0.0626627008120219}
499 Losses {'entity_linker': 0.5674233635266622}


In [59]:
doc = nlp(test_text)
for ent in doc.ents:
    if ent.label_ == 'persName' and ent.kb_id_ != 'NIL':
        print(ent.text, ent.label_, ent.kb_id_)

Iwan Groźny persName Q7996
Zygmunt August persName Q54058


Instalacja biblioteki wikibaseintegrator, do komunikacji z wikidata.org

In [ ]:
!pip install wikibaseintegrator

Niezbędne importy z bilbioteki i ustawienie parametru USER_AGENT, bez którego wikdata.org nie będzie chciała współpracować.

In [21]:
from wikibaseintegrator import wbi_helpers
from wikibaseintegrator import WikibaseIntegrator
from wikibaseintegrator.wbi_config import config as wbi_config

wbi_config['USER_AGENT'] = 'MyWikibaseBot/1.0 (https://www.wikidata.org/wiki/User:MyUsername)'

Funkcja wikilinker - proste wyszukiwanie postaci w wikidata.org (w polskiej wersji tego serwisu).

In [65]:
def wikilinker(search_entity:str, min_year: int, max_year: int, number_of_candidates=10) -> str:
    """ funkcja wyszukuje w wikidata.org identyfikator najlepiej pasujący do postaci """
    wbi = WikibaseIntegrator()
    lista_qid = wbi_helpers.search_entities(search_entity, language='pl', search_type='item', max_results=number_of_candidates, allow_anonymous=True)

    best_qid = ''
    for item_qid in lista_qid:
        my_item = wbi.item.get(entity_id=item_qid)
        label = my_item.labels.get(language='pl')
        claims = my_item.claims.claims
        if 'P31' in claims:
            list_instance_of = claims.get('P31')
            for item_instance_of in list_instance_of:
                instance_of_value = item_instance_of.mainsnak.datavalue['value']['id']
                if instance_of_value == 'Q5':
                    if 'P570' in claims:
                        list_date_death = claims.get('P570')
                        for item_date_death in list_date_death:
                            date_death_value = item_date_death.mainsnak.datavalue['value']['time']
                            if len(date_death_value) > 5:
                                year = date_death_value[1:5]
                                if year.isdigit():
                                    year_int = int(year)
                                    if year_int >= min_year and year_int <= max_year:
                                        best_qid = item_qid
                                        break

    return best_qid
                                

Test funkcji wikilinker:

In [60]:
wikilinker('Iwan Groźny', 1501, 1600)

'Q7996'

In [62]:
test = """Już od czerwca bawił król Zygmunt August w pomorskiej  ziemi, lipiec i sierpień spędził w Gdańsku, 
we wrześniu po kilkudniowym  pobycie w Malborgu w czasie sejmiku podążył do Królewca na zaproszenie ks. Albrechta. 
Towarzyszyli mu w podróży hetman Jan Tarnowski , marszałek koronny Piotr Kmita, bp. Stanisław Hozjusz, kanclerz 
Jan Ocieski, podkanclerzy Jan Przerębski i inni przedniejsi senatorowie i dostojnicy polscy obok przedstawicieli 
świata umysłowego w osobach Marcina Kromera, Szymona Maricjusa-Czystochlebskiego i Łukasza Górnickiego."""

In [69]:
doc = nlp(test)
for ent in doc.ents:
    if ent.label_ == 'persName':
        person = ent.lemma_
        candidate_qid = wikilinker(person, 1550, 1625)
        print(ent.text, ent.label_, candidate_qid)

Zygmunt August persName Q54058
Albrechta persName Q40433
Jan Tarnowski persName Q970324
Piotr Kmita persName Q355998
Stanisław Hozjusz persName Q61962
Jan Ocieski persName Q12137032
Jan Przerębski persName Q11718767
Marcina Kromera persName Q66649
Szymona Maricjusa-Czystochlebskiego persName 
Łukasza Górnickiego persName Q345583


W 9 przypadkach na 10, prosta funkcja wikilinker znalazła prawidłowy identyfikator postaci w wikidata.org, udało 
się to nawet dla imienia 'Albrechta', zapewne trochę przypadkiem dla podanego zakresu lat (zakresu dla daty śmierci postaci przyjętego na lata 1550-1625 ze względu na znaną tematykę przetwarzanego artykułu) i imienia pierwszy znaleziny element to właśnie Albrecht Hohenzollern. Jedyna nieznaleziona postać to Szymon Maricjus-Czystochlebski - prawdopodobnie jest to Q9352684 (Szymon Marycjusz, alias: Szymon Maricjusz z Pilzna) ale różnica między encją NER w tekście a etykietą w wikidata.org jest zbyt duża by idnetyfikator mógł zostać znaleziony.  